In [1]:
%config Completer.use_jedi = False
%matplotlib inline

#!/usr/bin/env python
import sys
import threading
import time

import cv2 as cv
import numpy as np

from centermask.config import get_cfg
from detectron2.data import MetadataCatalog
from cv_bridge import CvBridge, CvBridgeError

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2_ros.msg import Result
from sensor_msgs.msg import Image, RegionOfInterest, CameraInfo
import pyrealsense2 as rs2

# from detectron2_ros.msg import PersonPositions, PersonPosition
from finean_msgs.msg import PersonPositions, PersonPosition

# For sync depth and rgb
import message_filters


# Needed to update backbone registry
import centermask.modeling.backbone


# For the dataset loading
from os import listdir
from os.path import isfile, join
import csv

from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
class DatasetAnalyser(object):
    def __init__(self, depth_offset=100, camera_type='hsr'):
        print("Initializing")
        setup_logger()

        self._bridge = CvBridge()
        self.score_thresh = 0.50
        self.removal_classes = ['person']
        self.camera_type = camera_type
        
        self.cfg = get_cfg()
        self.cfg.merge_from_file("/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml")
        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 # set threshold for this model
        self.cfg.MODEL.WEIGHTS = "/root/centermask2-lite-V-39-eSE-FPN-ms-4x.pth"
        self.predictor = DefaultPredictor(self.cfg)
        self._class_names = MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]).get("thing_classes", None)

        if camera_type == 'hsr':
            self.getManualHSRIntrinsics()
        elif camera_type == 'realsense':
            self.getManualRealsenseIntrinsics()
        else:
            print("Please specify a valid camera type.")
            return
            
        self.depth_offset = depth_offset
        
        print("Centermask2 Node Initialized")

    def convertPixelToPosition(self, depth_img, mask):
        # Get the pixel location and depth of person centroid

#         mask_positions_2d = np.argwhere(mask==True)
# #         print("2d mask")
# #         print(mask_positions_2d)
#         approx_head_thresh = np.quantile(mask_positions_2d[:,0], 1.0)
# #         print("approx_head_thresh")
# #         print(approx_head_thresh)
#         head_pos_mask = mask_positions_2d[:,0] < approx_head_thresh
# #         print("head_pos_mask")
# #         print(head_pos_mask)
        
#         count = (head_pos_mask == 1).sum()

#         y_center, x_center = mask_positions_2d[head_pos_mask, :].sum(0)/count
        
#         y_thresh_mask = (np.argwhere((mask == True) | (mask == False))[:,0] < approx_head_thresh)
#         y_thresh_mask = y_thresh_mask.reshape((480,640))
        
#         head_mask = mask & y_thresh_mask

        mask_positions_2d = np.argwhere(mask==True)
        
#         print(np.median(mask_positions_2d))
        y_center, x_center = np.mean(mask_positions_2d, axis=0)
        
        head_mask = mask
        
        
        non_zero_depth = depth_img[head_mask.astype(bool)]
        non_zero_depth = non_zero_depth[non_zero_depth>0]
        depth = np.median(non_zero_depth) + self.depth_offset

        # Convert to 3D position in camera coords 
        xyz = rs2.rs2_deproject_pixel_to_point(self.intrinsics, [x_center, y_center], depth/1000.0)
        
        return xyz
    
    def getManualHSRIntrinsics(self):
        self.intrinsics = rs2.intrinsics()
        self.intrinsics.width = 640
        self.intrinsics.height = 480
        self.intrinsics.ppx = 322.2156457958147
        self.intrinsics.ppy = 238.8396597454201
        self.intrinsics.fx = 537.3372465571922
        self.intrinsics.fy = 537.8279138435478
        self.intrinsics.model = rs2.distortion.brown_conrady

    def getManualRealsenseIntrinsics(self):
        
        self.intrinsics = rs2.intrinsics()
        self.intrinsics.width = 640
        self.intrinsics.height = 480
        self.intrinsics.ppx = 324.9241638183594
        self.intrinsics.ppy = 236.38864135742188
        self.intrinsics.fx = 618.6040649414062
        self.intrinsics.fy = 618.9261474609375
        self.intrinsics.model = rs2.distortion.brown_conrady

    def run(self, rgb_file, depth_file, camera_frame_transform):

        # Convert images
        np_image = cv.imread(rgb_file)
        np_depth_image = cv.imread(depth_file, cv.IMREAD_ANYDEPTH)

        np_depth_image_copy = np.zeros(np_depth_image.shape)
        
        # Get the masks and objects seen
        outputs = self.predictor(np_image)
        result = outputs["instances"].to("cpu")
        
#         result_msg = self.getResult(result)
        
        # Mask out people with score greater than threshold
        class_ids = result.pred_classes if result.has("pred_classes") else None
        class_names = np.array(self._class_names)[class_ids.numpy()]        
        retain_inds = (result.scores.numpy() > self.score_thresh) & (class_names == 'person')
        result = result[retain_inds]

        num_inds = sum(retain_inds)
#         print("Number of masks: {0}".format(num_inds))

#         boxes = result.pred_boxes if result.has("pred_boxes") else None
#         for i, (x1, y1, x2, y2) in enumerate(boxes):
#             continue
    
        positions_3d_world = []
        if num_inds > 0:
            masks = np.asarray(result.pred_masks)
            for i in range(num_inds):
                mask = masks[i]
#                 print(result)
            
                if sum(sum(mask == True)) <=10:
                    continue
            
#                 print("Lenght of mask: " + str(len(mask)))
                person_position = self.convertPixelToPosition(np_depth_image, mask)
#                 print(person_position)
                np_depth_image_copy[mask] = 1
                
                # Convert to world frame                 
                head_T_person = np.eye(4)
                head_T_person[0:3,3] = [person_position[0], person_position[1], person_position[2]]
                
                world_T_person = np.dot(camera_frame_transform, head_T_person)

#                 print("head_T_person")
#                 print(head_T_person)
                world_xyz = world_T_person[0:3,3]

#                 print(world_xyz)
                positions_3d_world += list(world_xyz)
        
#         plt.imshow(np_depth_image_copy)
      
#         v = Visualizer(np_image[:, :, ::-1], MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]), scale=1.2)
#         v = v.draw_instance_predictions(result)
#         img = v.get_image()[:, :, ::-1]
#         plt.imshow(img)
            
#         print("Allpositions extracted...")
        return positions_3d_world
    
    def getResult(self, predictions):

        boxes = predictions.pred_boxes if predictions.has("pred_boxes") else None

        if predictions.has("pred_masks"):
            masks = np.asarray(predictions.pred_masks)
        else:
            return

        result_msg = Result()
        result_msg.class_ids = predictions.pred_classes if predictions.has("pred_classes") else None
        result_msg.class_names = np.array(self._class_names)[result_msg.class_ids.numpy()]
        result_msg.scores = predictions.scores if predictions.has("scores") else None

        for i, (x1, y1, x2, y2) in enumerate(boxes):
            mask = np.zeros(masks[i].shape, dtype="uint8")
            mask[masks[i, :, :]]=255
            mask = self._bridge.cv2_to_imgmsg(mask)
            result_msg.masks.append(mask)

            box = RegionOfInterest()
            box.x_offset = np.uint32(x1)
            box.y_offset = np.uint32(y1)
            box.height = np.uint32(y2 - y1)
            box.width = np.uint32(x2 - x1)
            result_msg.boxes.append(box)

        return result_msg
    

    @staticmethod
    def load_param(param, default=None):
        new_param = rospy.get_param(param, default)
        rospy.loginfo("[Centermask2] %s: %s", param, new_param)
        return new_param

In [ ]:
import csv
import numpy
from glob import glob

dataset_folder = "/root/HumanTrajectoryPredictionDataset"

camera_transform_dir = dataset_folder + "/formatted"

output_positions_dir = dataset_folder + "/estimated_positions"

hsr_analyser = DatasetAnalyser(depth_offset=125, camera_type='hsr')
realsense_analyser = DatasetAnalyser(depth_offset=125, camera_type='realsense')

for map_num in [1,2,3,4]:
    for run in [1,2,3]:
# for map_num in [1]:
#     for run in [1]:
        hsr_rgb_image_folder = dataset_folder + "/images/hsr/map{0}_run{1}/rgb".format(map_num, run)
        hsr_depth_image_folder = dataset_folder + "/images/hsr/map{0}_run{1}/depth".format(map_num, run)
        realsense_rgb_image_folder = dataset_folder + "/images/realsense/map{0}_run{1}/rgb".format(map_num, run)
        realsense_depth_image_folder = dataset_folder + "/images/realsense/map{0}_run{1}/depth".format(map_num, run)

        hsr_camera_transform_file = camera_transform_dir + "/map{0}_run{1}_hsr.csv".format(map_num, run)
        realsense_camera_transform_file = camera_transform_dir + "/map{0}_run{1}_realsense.csv".format(map_num, run)
        
        hsr_reader = csv.reader(open(hsr_camera_transform_file), delimiter=",")
        realsense_reader = csv.reader(open(realsense_camera_transform_file), delimiter=",")
        
        hsr_transform_list = list(hsr_reader)
        realsense_transform_list = list(realsense_reader)
        
        hsr_position_results = []
        
        hsr_depth_tstamps = [row.split('/')[-1].strip('.png') for row in glob(hsr_depth_image_folder + '/*.png')]
        hsr_depth_tstamps_floats = np.array([float(i) for i in hsr_depth_tstamps])

        realsense_depth_tstamps = [row.split('/')[-1].strip('.png') for row in glob(realsense_depth_image_folder + '/*.png')]
        realsense_depth_tstamps_floats = np.array([float(i) for i in realsense_depth_tstamps])
        
        for row in tqdm(hsr_transform_list):
            t_stamp = row[0]
            
            depth_t_str = hsr_depth_tstamps[np.argmin(abs(hsr_depth_tstamps_floats - float(t_stamp)))]
            
            rgb_file = hsr_rgb_image_folder + "/{0}.jpg".format(t_stamp)
            depth_file = hsr_depth_image_folder + "/{0}.png".format(depth_t_str)

            hsr_camera_transform = np.array(row[1::]).astype(float).reshape((4,4))
            hsr_human_poses = hsr_analyser.run(rgb_file, 
                                               depth_file, 
                                               hsr_camera_transform)
        
            hsr_position_results.append([t_stamp] + hsr_human_poses)

        with open(output_positions_dir + '/map{0}_run{1}_hsr.csv'.format(map_num, run), 'w') as myhsrfile:
            hsr_wr = csv.writer(myhsrfile, quoting=csv.QUOTE_ALL)
            for i in range(len(hsr_position_results)):
                hsr_wr.writerow(hsr_position_results[i])
                
                
                
                
                
        realsense_position_results = []
        for row in tqdm(realsense_transform_list):

            t_stamp = row[0]

            depth_t_str = realsense_depth_tstamps[np.argmin(abs(realsense_depth_tstamps_floats - float(t_stamp)))]

            rgb_file = realsense_rgb_image_folder + "/{0}.jpg".format(t_stamp)
            depth_file = realsense_depth_image_folder + "/{0}.png".format(depth_t_str)
            
            realsense_camera_transform = np.array(row[1::]).astype(float).reshape((4,4))
            realsense_human_poses = realsense_analyser.run(rgb_file, 
                                               depth_file, 
                                               realsense_camera_transform)
            
            realsense_position_results.append([t_stamp] + realsense_human_poses)
            

        with open(output_positions_dir + '/map{0}_run{1}_realsense.csv'.format(map_num, run), 'w') as myrealsensefile:
            realsense_wr = csv.writer(myrealsensefile, quoting=csv.QUOTE_ALL)
            for i in range(len(realsense_position_results)):
                realsense_wr.writerow(realsense_position_results[i])
    
        
        print("Finished mapnum:{0} \t run:{1}".format(map_num, run))
# result = numpy.array(x).astype("float")

#         for t_stamp in 
#         [hsr_result, hsr_poses] = hsr_analyser.run(rgb_file, depth_file, hsr_camera_transform)


In [ ]:
hsr_analyser = DatasetAnalyser(depth_offset=125, camera_type='hsr')
hsr_human_poses = hsr_analyser.run(rgb_file, 
                                   depth_file, 
                                   hsr_camera_transform)

In [ ]:
realsense_depth_image_folder

In [ ]:
with open(output_positions_dir + '/map{0}_run{1}_hsr.csv'.format(map_num, run), 'w') as myhsrfile:
    hsr_wr = csv.writer(myhsrfile, quoting=csv.QUOTE_ALL)
    for i in range(len(hsr_position_results)):
        hsr_wr.writerow(hsr_position_results[i])

In [ ]:
hsr_transform_list[0]
np.array(hsr_transform_list[0][1::]).astype(float).reshape((4,4))

In [ ]:
hsr_transform_list[0]

In [ ]:
hsr_analyser = DatasetAnalyser(depth_offset=125)
        
row = hsr_transform_list[0]
t_stamp = row[0]

depth_t_str = depth_tstamps[np.argmin(depth_tstamps_floats - float(t_stamp))]

rgb_file = hsr_rgb_image_folder + "/{0}.jpg".format(t_stamp)
depth_file = hsr_depth_image_folder + "/{0}.png".format(depth_t_str)

hsr_camera_transform = np.array(row[1::]).astype(float).reshape((4,4))
hsr_human_poses = hsr_analyser.run(rgb_file, 
                                   depth_file, 
                                   hsr_camera_transform)

In [ ]:
plt.imshow(cv.imread(rgb_file))

# isolated testing

In [ ]:
import csv
import numpy
from glob import glob

dataset_folder = "/root/HumanTrajectoryPredictionDataset"

camera_transform_dir = dataset_folder + "/formatted"

output_positions_dir = dataset_folder + "/estimated_positions"

hsr_analyser = DatasetAnalyser(depth_offset=125, camera_type='hsr')
realsense_analyser = DatasetAnalyser(depth_offset=125, camera_type='realsense')


map_num = 1
run = 1

hsr_rgb_image_folder = dataset_folder + "/images/hsr/map{0}_run{1}/rgb".format(map_num, run)
hsr_depth_image_folder = dataset_folder + "/images/hsr/map{0}_run{1}/depth".format(map_num, run)
realsense_rgb_image_folder = dataset_folder + "/images/realsense/map{0}_run{1}/rgb".format(map_num, run)
realsense_depth_image_folder = dataset_folder + "/images/realsense/map{0}_run{1}/depth".format(map_num, run)

hsr_camera_transform_file = camera_transform_dir + "/map{0}_run{1}_hsr.csv".format(map_num, run)
realsense_camera_transform_file = camera_transform_dir + "/map{0}_run{1}_realsense.csv".format(map_num, run)

hsr_reader = csv.reader(open(hsr_camera_transform_file), delimiter=",")
realsense_reader = csv.reader(open(realsense_camera_transform_file), delimiter=",")

hsr_transform_list = list(hsr_reader)
realsense_transform_list = list(realsense_reader)

hsr_position_results = []

hsr_depth_tstamps = [row.split('/')[-1].strip('.png') for row in glob(hsr_depth_image_folder + '/*.png')]
hsr_depth_tstamps_floats = np.array([float(i) for i in hsr_depth_tstamps])

realsense_depth_tstamps = [row.split('/')[-1].strip('.png') for row in glob(realsense_depth_image_folder + '/*.png')]
realsense_depth_tstamps_floats = np.array([float(i) for i in realsense_depth_tstamps])

In [ ]:
for row in tqdm(hsr_transform_list[0:250]):
    t_stamp = row[0]

    depth_t_str = hsr_depth_tstamps[np.argmin(hsr_depth_tstamps_floats - float(t_stamp))]

    rgb_file = hsr_rgb_image_folder + "/{0}.jpg".format(t_stamp)
    depth_file = hsr_depth_image_folder + "/{0}.png".format(depth_t_str)

    hsr_camera_transform = np.array(row[1::]).astype(float).reshape((4,4))
    hsr_human_poses = hsr_analyser.run(rgb_file, 
                                       depth_file, 
                                       hsr_camera_transform)

    hsr_position_results.append([t_stamp] + hsr_human_poses)

# with open(output_positions_dir + '/map{0}_run{1}_hsr.csv'.format(map_num, run), 'w') as myhsrfile:
#     hsr_wr = csv.writer(myhsrfile, quoting=csv.QUOTE_ALL)
#     for i in range(len(hsr_position_results)):
#         hsr_wr.writerow(hsr_position_results[i])

In [ ]:
hsr_analyser = DatasetAnalyser(depth_offset=100, camera_type='hsr')

row = hsr_transform_list[312]
t_stamp = row[0]

depth_t_str = hsr_depth_tstamps[np.argmin(abs(hsr_depth_tstamps_floats - float(t_stamp)))]

rgb_file = hsr_rgb_image_folder + "/{0}.jpg".format(t_stamp)
depth_file = hsr_depth_image_folder + "/{0}.png".format(depth_t_str)

hsr_camera_transform = np.array(row[1::]).astype(float).reshape((4,4))
hsr_human_poses = hsr_analyser.run(rgb_file, 
                                   depth_file, 
                                   hsr_camera_transform)


In [ ]:
hsr_human_poses

In [ ]:
plt.imshow(cv.imread(rgb_file))

In [ ]:
plt.imshow(plt.imread(depth_file))

In [ ]:
realsense_analyser = DatasetAnalyser(depth_offset=125, camera_type='realsense')
row = realsense_transform_list[327]
t_stamp = row[0]

depth_t_str = realsense_depth_tstamps[np.argmin(abs(realsense_depth_tstamps_floats - float(t_stamp)))]

rgb_file = realsense_rgb_image_folder + "/{0}.jpg".format(t_stamp)
depth_file = realsense_depth_image_folder + "/{0}.png".format(depth_t_str)

realsense_camera_transform = np.array(row[1::]).astype(float).reshape((4,4))
realsense_human_poses = realsense_analyser.run(rgb_file, 
                                   depth_file, 
                                   realsense_camera_transform)

In [ ]:
realsense_human_poses

In [ ]:
-1.7424; 0.54158; 1.859

In [ ]:
plt.imshow(np.asarray(result.pred_masks).astype(float)[0])

In [ ]:
non_zero_depth = non_zero_depth[non_zero_depth>0]

np.median(np_depth_image[np.asarray(result.pred_masks)[0]])

In [ ]:
np_depth_image = cv.imread(depth_file, cv.IMREAD_ANYDEPTH)
plt.imshow(np_depth_image)

In [ ]:
np_depth_image[100,590]

# Offset testing

In [3]:
import os

def checkMakeDir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [4]:
import csv
import numpy
from glob import glob

dataset_folder = "/root/HumanTrajectoryPredictionDataset"

camera_transform_dir = dataset_folder + "/formatted"

# output_positions_dir = dataset_folder + "/estimated_positions"
# checkMakeDir(output_positions_dir)

# for depth_offset in [0, 50, 100, 150, 200, 250, 300]:
for depth_offset in [150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250]:
    hsr_analyser = DatasetAnalyser(depth_offset=depth_offset, camera_type='hsr')
    realsense_analyser = DatasetAnalyser(depth_offset=depth_offset, camera_type='realsense')
#     output_positions_dir = dataset_folder + "/estimated_positions/offset_{0}".format(depth_offset) 
    output_positions_dir = dataset_folder + "/finetuned_positions/offset_{0}".format(depth_offset) 
    checkMakeDir(output_positions_dir)
    
#     for map_num in [1,2,3,4]:
#         for run in [1,2,3]:
    for map_num in [1]:
        for run in [1]:
            hsr_rgb_image_folder = dataset_folder + "/images/hsr/map{0}_run{1}/rgb".format(map_num, run)
            hsr_depth_image_folder = dataset_folder + "/images/hsr/map{0}_run{1}/depth".format(map_num, run)
            realsense_rgb_image_folder = dataset_folder + "/images/realsense/map{0}_run{1}/rgb".format(map_num, run)
            realsense_depth_image_folder = dataset_folder + "/images/realsense/map{0}_run{1}/depth".format(map_num, run)

            hsr_camera_transform_file = camera_transform_dir + "/map{0}_run{1}_hsr.csv".format(map_num, run)
            realsense_camera_transform_file = camera_transform_dir + "/map{0}_run{1}_realsense.csv".format(map_num, run)

            hsr_reader = csv.reader(open(hsr_camera_transform_file), delimiter=",")
            realsense_reader = csv.reader(open(realsense_camera_transform_file), delimiter=",")

            hsr_transform_list = list(hsr_reader)
            realsense_transform_list = list(realsense_reader)

            hsr_position_results = []

            hsr_depth_tstamps = [row.split('/')[-1].strip('.png') for row in glob(hsr_depth_image_folder + '/*.png')]
            hsr_depth_tstamps_floats = np.array([float(i) for i in hsr_depth_tstamps])

            realsense_depth_tstamps = [row.split('/')[-1].strip('.png') for row in glob(realsense_depth_image_folder + '/*.png')]
            realsense_depth_tstamps_floats = np.array([float(i) for i in realsense_depth_tstamps])

            for row in tqdm(hsr_transform_list):
                t_stamp = row[0]

                depth_t_str = hsr_depth_tstamps[np.argmin(abs(hsr_depth_tstamps_floats - float(t_stamp)))]

                rgb_file = hsr_rgb_image_folder + "/{0}.jpg".format(t_stamp)
                depth_file = hsr_depth_image_folder + "/{0}.png".format(depth_t_str)

                hsr_camera_transform = np.array(row[1::]).astype(float).reshape((4,4))
                hsr_human_poses = hsr_analyser.run(rgb_file, 
                                                   depth_file, 
                                                   hsr_camera_transform)

                hsr_position_results.append([t_stamp] + hsr_human_poses)

            with open(output_positions_dir + '/map{0}_run{1}_hsr.csv'.format(map_num, run), 'w') as myhsrfile:
                hsr_wr = csv.writer(myhsrfile, quoting=csv.QUOTE_ALL)
                for i in range(len(hsr_position_results)):
                    hsr_wr.writerow(hsr_position_results[i])





            realsense_position_results = []
            for row in tqdm(realsense_transform_list):

                t_stamp = row[0]

                depth_t_str = realsense_depth_tstamps[np.argmin(abs(realsense_depth_tstamps_floats - float(t_stamp)))]

                rgb_file = realsense_rgb_image_folder + "/{0}.jpg".format(t_stamp)
                depth_file = realsense_depth_image_folder + "/{0}.png".format(depth_t_str)

                realsense_camera_transform = np.array(row[1::]).astype(float).reshape((4,4))
                realsense_human_poses = realsense_analyser.run(rgb_file, 
                                                   depth_file, 
                                                   realsense_camera_transform)

                realsense_position_results.append([t_stamp] + realsense_human_poses)


            with open(output_positions_dir + '/map{0}_run{1}_realsense.csv'.format(map_num, run), 'w') as myrealsensefile:
                realsense_wr = csv.writer(myrealsensefile, quoting=csv.QUOTE_ALL)
                for i in range(len(realsense_position_results)):
                    realsense_wr.writerow(realsense_position_results[i])


            print("Finished mapnum:{0} \t run:{1}".format(map_num, run))
    # result = numpy.array(x).astype("float")

    #         for t_stamp in 
    #         [hsr_result, hsr_poses] = hsr_analyser.run(rgb_file, depth_file, hsr_camera_transform)


Initializing
WARNING [10/31 10:32:10 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized
Initializing
WARNING [10/31 10:32:14 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized


  6%|▋         | 591/9442 [00:39<09:52, 14.95it/s]/root/.virtualenvs/centermask2/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/root/.virtualenvs/centermask2/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 9483/9483 [10:25<00:00, 15.15it/s]


Finished mapnum:1 	 run:1
Initializing
WARNING [10/31 10:53:12 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized
Initializing
WARNING [10/31 10:53:12 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized


100%|██████████| 9483/9483 [10:22<00:00, 15.24it/s]


Finished mapnum:1 	 run:1
Initializing
WARNING [10/31 11:14:01 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized
Initializing
WARNING [10/31 11:14:01 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized


...

Finished mapnum:1 	 run:1
Initializing
WARNING [10/31 13:40:13 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized
Initializing
WARNING [10/31 13:40:14 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized


100%|██████████| 9483/9483 [10:30<00:00, 15.04it/s]


Finished mapnum:1 	 run:1
Initializing
WARNING [10/31 14:01:15 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized
Initializing
WARNING [10/31 14:01:16 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Centermask2 Node Initialized


100%|██████████| 9483/9483 [10:29<00:00, 15.07it/s]

Finished mapnum:1 	 run:1
